In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use(['science', 'no-latex'])

from pathlib import Path

p = Path(".")
files = list(p.resolve().rglob("0*.jsonl"))
print(files)
df = pd.concat([pd.read_json(f, orient="records", lines=True) for f in files], ignore_index=True)

from sklearn.metrics import accuracy_score, f1_score

df["accuracy"] = df.apply(
    lambda row: accuracy_score(row["y_true"], row["y_pred"]), axis=1
)
df["macroF1"] = df.apply(
    lambda row: f1_score(row["y_true"], row["y_pred"], average="macro"), axis=1
)

def assign_model(row):
    try:
        return row["train_config"]["model_name_or_path"]
    except:
        return "/".join(row["model_name_or_path"].split("/")[-2].split("_")[0:2])

df["model"] = df.apply(assign_model, axis=1)

[PosixPath('/home/peterr/macocu/task11/010_results.jsonl'), PosixPath('/home/peterr/macocu/task11/011_results.jsonl'), PosixPath('/home/peterr/macocu/task11/013_results_english.jsonl'), PosixPath('/home/peterr/macocu/task11/008_results.jsonl'), PosixPath('/home/peterr/macocu/task11/012_results_nonslavic.jsonl'), PosixPath('/home/peterr/macocu/task11/012_results.jsonl'), PosixPath('/home/peterr/macocu/task11/013_results_slavic_asr.jsonl')]


In [4]:
df.shape

(95, 10)

In [5]:
df.head(2)

,output_column,model_name_or_path,eval_file,clip_seconds,y_true,y_pred,train_config,accuracy,macroF1,model
0,Speaker_gender,models/facebook_wav2vec2-large-960h-lv60-self_...,001_gender_dev.csv,-1,"[F, F, F, F, F, F, F, F, F, F, F, F, F, F, F, ...","[F, F, F, F, F, F, F, F, F, F, F, F, F, F, F, ...",NaN,1.000,1.000,facebook/wav2vec2-large-960h-lv60-self
1,Speaker_gender,models/facebook_wav2vec2-large-960h-lv60-self_...,001_gender_test.csv,-1,"[F, F, F, F, F, F, F, F, F, F, F, F, F, F, F, ...","[F, F, F, F, F, F, F, F, F, F, F, F, F, F, F, ...",NaN,0.999,0.999,facebook/wav2vec2-large-960h-lv60-self


In [8]:
df = df.drop_duplicates(
    subset='output_column model_name_or_path eval_file clip_seconds accuracy macroF1'.split()
    )
df.shape

(36, 10)

In [19]:
df[df.model_name_or_path.str.contains('facebook_wav2vec2-large')].groupby("output_column model_name_or_path eval_file".split()).agg({"accuracy": 'mean'})

accuracy
output_column     model_name_or_path                                 eval_file                                     
Party_status      /home/peterr/macocu/task11/models/facebook_wav2... 012_dev.csv                           0.606667
                                                                     012_test.csv                          0.548889
                  /home/peterr/macocu/task11/models/facebook_wav2... 012_dev.csv                           0.613333
                                                                     012_test.csv                          0.590000
Speaker_age_group models/facebook_wav2vec2-large-960h-lv60-self_N... 006_age_dev.csv                       0.716000
                                                                     006_age_test.csv                      0.678000
                  models/facebook_wav2vec2-large-slavic-voxpopuli... 006_age_dev.csv                       0.704000
                                                                     006_age_test.csv                      0.694000
Speaker_gender    models/facebook_wav2vec2-large-960h-lv60-self_N... 001_gender_dev.csv                    1.000000
                                                                     001_gender_test.csv                   0.999000
                  models/facebook_wav2vec2-large-960h-lv60-self_N... 001_gender_dev.csv                    0.997000
                                                                     001_gender_test.csv                   0.993500
                  models/facebook_wav2vec2-large-slavic-voxpopuli... 001_gender_dev.csv                    0.994000
                                                                     001_gender_test.csv                   0.997000
                  models/facebook_wav2vec2-large-slavic-voxpopuli... 001_gender_dev.csv                    1.000000
                                                                     001_gender_test.csv                   0.989500
Speaker_name      models/facebook_wav2vec2-large-960h-lv60-self_N... 003_speaker_id_dev_for_datasets.csv   0.316000
                                                                     003_speaker_id_test_for_datasets.csv  0.334000
                  models/facebook_wav2vec2-large-960h-lv60-self_N... 003_speaker_id_dev_for_datasets.csv   0.140000
                                                                     003_speaker_id_test_for_datasets.csv  0.106000
                  models/facebook_wav2vec2-large-slavic-voxpopuli... 003_speaker_id_dev_for_datasets.csv   0.992000
                                                                     003_speaker_id_test_for_datasets.csv  0.998000
                  models/facebook_wav2vec2-large-slavic-voxpopuli... 003_speaker_id_dev_for_datasets.csv   0.738000
                                                                     003_speaker_id_test_for_datasets.csv  0.806000

In [16]:
df.model.unique()

array(['facebook/wav2vec2-large-960h-lv60-self',
       'facebook/wav2vec2-large',
       'facebook/wav2vec2-large-slavic-voxpopuli-v2',
       'classla/wav2vec2-large-slavic-parlaspeech-hr'], dtype=object)

In [21]:
df[df.model_name_or_path.str.contains('facebook_wav2vec2-large_')]

,output_column,model_name_or_path,eval_file,clip_seconds,y_true,y_pred,train_config,accuracy,macroF1,model
